<a href="https://colab.research.google.com/github/ignaciomz/OCR_fromImageLink/blob/main/Prueba_OCR_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract pillow openpyxl
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-spa


In [ ]:
import pytesseract
from PIL import Image
import requests
from io import BytesIO
from openpyxl import load_workbook
from urllib.parse import urlparse
from google.colab import files
import os


os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata"
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'


def es_url_valida(url):
    try:
        resultado = urlparse(url)
        return all([resultado.scheme, resultado.netloc])
    except:
        return False

# Cargar archivo Excel
print("Sube tu archivo Excel:")
uploaded = files.upload()
archivo_excel = list(uploaded.keys())[0]


wb = load_workbook(archivo_excel)
hoja = wb.active

# Columnas específicas (Aquí colocar el numero de la columna de tu archivo excel)
columna_links = X  # Numero de Columna con Link de Imagen
columna_resultados = X  # Numero de Columna donde alojará el resultado

# Procesar cada fila
for fila in range(2, hoja.max_row + 1):  # Empieza en la fila 2 (asumiendo encabezado en fila 1)
    celda_link = hoja.cell(row=fila, column=columna_links)
    celda_resultado = hoja.cell(row=fila, column=columna_resultados)

    link = celda_link.value
    if link and es_url_valida(link):
        try:
            print(f"Procesando fila {fila}/{hoja.max_row}")


            response = requests.get(link)
            response.raise_for_status()
            img = Image.open(BytesIO(response.content))

            # Realizar OCR
            texto = pytesseract.image_to_string(img, lang="spa")
            celda_resultado.value = texto.strip()

        except Exception as e:
            print(f"Error procesando la fila {fila}: {e}")
            celda_resultado.value = f"Error: {e}"
    else:
        celda_resultado.value = "Enlace no válido"

# Guardar los cambios en el archivo Excel
wb.save("OCR_Procesado.xlsx")
print("Proceso completado. Resultados guardados en 'OCR_Procesado.xlsx'.")

# Descarga automática del archivo
files.download("OCR_Procesado.xlsx")
